In [ ]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
from pathlib import Path
import numpy as np
import PIL
import PIL.Image
import json

#change working directory to root
ROOT_DIR = os.getcwd()
while os.path.basename(ROOT_DIR) != 'VisIrNet':
    ROOT_DIR = os.path.abspath(os.path.join(ROOT_DIR,'..'))
sys.path.insert(0,ROOT_DIR)
os.chdir(ROOT_DIR)

ROOT_DIR = Path(ROOT_DIR)

print(tf.__version__)
devices = tf.config.list_physical_devices('GPU')
print("len(devices): ", len(devices))
print(f"available GPUs: {devices}");


**Dataloaders**

In [ ]:
import data_setup

# try to import the dataset
dataset="SkyData"
BATCH_SIZE = 2
SHUFFLE_BUFFER_SIZE = 100

train_dataloader,test_dataloader = data_setup.create_dataloaders(dataset=dataset, 
                                                                BATCH_SIZE=BATCH_SIZE,
                                                                SHUFFLE_BUFFER_SIZE=100
                                                                )

len(train_dataloader), len(test_dataloader)
#

## **Model**

In [ ]:
import model_setup
import Utils

**configuration**


or load configuration from a file

In [ ]:
# config constants 
configs = {
            'RGB_INPUTS_SHAPE' : (192,192,3),
            'IR_INPUTS_SHAPE' :  (128,128,3),
            'OUTPUT_CHANNELS_PER_BLOCK' : 3,
            'REGRESSION_INPUT_SHAPE' : None,
            'REGRESSION_OUTPUT_SHAPE' : 8    
            }
configs['REGRESSION_INPUT_SHAPE']= (*configs["RGB_INPUTS_SHAPE"][:2], configs["OUTPUT_CHANNELS_PER_BLOCK"]*2)
assert configs['REGRESSION_INPUT_SHAPE'] != None

In [ ]:
featureEmbeddingBackBone = model_setup.getFeatureEmbeddingBackBone(rgb_inputs_shape=configs['RGB_INPUTS_SHAPE'],
                                                        ir_inputs_shape=configs['IR_INPUTS_SHAPE'],
                                                        output_channels_per_block=configs['OUTPUT_CHANNELS_PER_BLOCK']
                                                        )

regressionHead= model_setup.getRegressionHead(input_shape=configs['REGRESSION_INPUT_SHAPE'],
                                                output_size=configs['REGRESSION_OUTPUT_SHAPE']
                                                )


**Visualize and save model structures**

In [ ]:
# # visualize and save models

# Utils.plot_and_save_model_structure(featureEmbeddingBackBone,
#                                             save_path="resources/",
#                                             save_as=f"featureEmbeddingBackBone")
# Utils.plot_and_save_model_structure(regressionHead,
#                                             save_path="resources/",
#                                             save_as=f"regressionHead")

## **Training**


**first stage**

In [ ]:
# import engine 

# initial_learning_rate = 0.001
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,
#                                                                 decay_steps=10000,
#                                                                 decay_rate=0.96,
#                                                                 staircase=True)
# NUM_EPOCHS = 5

# # Setup optimizer
# optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# # Start the timer
# from timeit import default_timer as timer
# start_time = timer()
# # Train model 

# model_results = engine.train_first_stage(model=featureEmbeddingBackBone,
#                                                 train_dataloader=train_dataloader,
#                                                 test_dataloader=test_dataloader,
#                                                 optimizer=optimizer,
#                                                 epochs=NUM_EPOCHS,
#                                                 from_checkpoint="latest",
#                                                 save_path="models",
#                                                 save_as=f"featureEmbeddingBackBone",
#                                                 save_frequency=1,
#                                                 save_hard_frequency=50
#                                                 )
# # End the timer and print out how long it took
# end_time = timer()
# print(f"Total training time : {end_time-start_time:.3f} seconds")

**second stage**

In [ ]:
import engine 

initial_learning_rate = 0.0001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate,
                                                                decay_steps=10000,
                                                                decay_rate=0.96,
                                                                staircase=True)
NUM_EPOCHS = 2

# Setup optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Start the timer
from timeit import default_timer as timer
start_time = timer()
# Train model 

model_results = engine.train_second_stage(model=regressionHead,
                                        featureEmbeddingBackBone="latest",
                                        train_dataloader=train_dataloader,
                                        test_dataloader=test_dataloader,
                                        optimizer=optimizer,
                                        epochs=NUM_EPOCHS,
                                        from_checkpoint="latest",
                                        save_path="models",
                                        save_as=f"regressionHead",
                                        save_frequency=1,
                                        save_hard_frequency=20
                                        )
# End the timer and print out how long it took
end_time = timer()
print(f"Total training time : {end_time-start_time:.3f} seconds")

In [1]:
import tensorflow as tf

2023-10-09 17:13:24.780272: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-09 17:13:24.814875: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-09 17:13:24.814905: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-09 17:13:24.814926: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 17:13:24.820420: I tensorflow/core/platform/cpu_feature_g

In [14]:
t = tf.ones([1,192,192,3])
t2 = tf.ones([1,128,128,3])
t.shape, t2.shape

(TensorShape([1, 5, 5, 3]), TensorShape([1, 2, 2, 3]))

In [ ]:
pad_height = (t.shape[1] - t2.shape[1]) // 2
pad_width = (t.shape[2] - t2.shape[2]) // 2

In [15]:
paddings = [[0,0], [pad_height, pad_height], [pad_width, pad_width], [0,0]]
paddings

[[0, 0], [1, 1], [1, 1], [0, 0]]

In [18]:
t2

<tf.Tensor: shape=(1, 2, 2, 3), dtype=float32, numpy=
array([[[[ 0.46643844, -1.1350483 , -1.5372299 ],
         [-0.49728367, -0.50578505,  0.38425392]],

        [[-0.6790849 ,  1.1726139 , -0.7692692 ],
         [ 0.7509908 , -0.43396217,  0.7430156 ]]]], dtype=float32)>

In [17]:

out = tf.pad(t2, paddings, 'CONSTANT', constant_values=0)
out[0]

<tf.Tensor: shape=(4, 4, 3), dtype=float32, numpy=
array([[[ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ],
        [ 0.46643844, -1.1350483 , -1.5372299 ],
        [-0.49728367, -0.50578505,  0.38425392],
        [ 0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ],
        [-0.6790849 ,  1.1726139 , -0.7692692 ],
        [ 0.7509908 , -0.43396217,  0.7430156 ],
        [ 0.        ,  0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]], dtype=float32)>